## LangChain - Agents intro

Tools are a way augment our LLMs with code execution. A tool is simply a function formatted so that our agent can undertstand how to use it, and then execute it. Let's start by creating a few simple tools.

We can use the @tool decorator to create an LLM-compatible tool from a standard python function — this function should include a few things for optimal performance:
- A docstring describing what the tool does and when it should be used, this will be read by our LLM/agent and used to decide when to use the tool, and also how to use the tool.
- Clear parameter names that ideally tell the LLM what each parameter is, if it isn't clear we make sure the docstring explains what the parameter is for and how to use it.
- Both parameter and return type annotations.



### Creating tools

In [1]:
from langchain_core.tools import tool

In [2]:
@tool
def add(x: float, y: float) -> float:
    """
    Add 'x' and 'y'.
    """
    return x + y


@tool
def multiply(x: float, y: float) -> float:
    """
    Multiply 'x' and 'y'
    """
    return x * y


@tool
def exponentiate(x: float, y: float) -> float:
    """
    Raise 'x' to the power of 'y'
    """
    return x**y

@tool
def subtract(x: float, y: float) -> float:
    """
    Subtract 'x' from 'y'
    """
    return y - x

In [3]:
# With the @tool decorator our function is turned into a StructureTool object
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x000001F5DE7056C0>)

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the bellow

In [4]:
import json

In [5]:
llm_output_string = "{\"x\": 5, \"y\": 2}" # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string) # load as a dict
llm_output_dict

{'x': 5, 'y': 2}

In [6]:
exponentiate.func(**llm_output_dict)

25

### Tool calling agent

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [8]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [10]:
from langchain.memory import ConversationBufferMemory

In [11]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

C:\Users\BEU7CA\AppData\Local\Temp\ipykernel_26232\515166408.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [12]:
from langchain.agents import create_tool_calling_agent

In [13]:
tools = [add, subtract, multiply, exponentiate]

In [14]:
agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

In [ ]:
# Just called the tool, not executed it
agent.invoke({
    "input": "what is 10.7 multiplied by 7.68",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": [] # agent will append it's internal steps here
})

[ToolAgentAction(tool='multiply', tool_input={'x': 10.7, 'y': 7.68}, log="\nInvoking: `multiply` with `{'x': 10.7, 'y': 7.68}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9nu8M3MDZo63mEJ4gclS1LGj', 'function': {'arguments': '{"x":10.7,"y":7.68}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 145, 'total_tokens': 166, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C3RcP596VKHK75Ne4AxW2sjGNosPA', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--25bb2999-738e-43b0-b569-de7a4f3ded0c-0', tool_calls=[{'name': 'multiply', 'args': {'x': 10.7, 'y': 7.68}, 'id': 'call_9nu8M3

In [16]:
# To execute we need a Agent executor
from langchain.agents import AgentExecutor

In [17]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

In [ ]:
agent_executor.invoke({
"input": "what is 10.7 multiplied by 7.68?",
"chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10.7, 'y': 7.68}`


82.1759999999999910.7 multiplied by 7.68 is approximately 82.18.

> Finished chain.


{'input': 'what is 10.7 multiplied by 7.68?',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={})],
 'output': '10.7 multiplied by 7.68 is approximately 82.18.'}

In [19]:
10.7*7.68

82.17599999999999

In [20]:
agent_executor.invoke({
    "input": "My name is Luis",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Nice to meet you, Luis! How can I assist you today?

> Finished chain.


{'input': 'My name is Luis',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Luis', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Luis! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Nice to meet you, Luis! How can I assist you today?'}

In [21]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2, to the power of 3",
    "chat_history": memory
})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'x': 9, 'y': 10}`


19.0
Invoking: `multiply` with `{'x': 4, 'y': 2}`


8.0
Invoking: `exponentiate` with `{'x': 2, 'y': 3}`


8.0
Invoking: `subtract` with `{'x': 19, 'y': 8}`


-11.0The result of \( 9 + 10 - 4 \times 2^3 \) is \(-11\).

> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2, to the power of 3',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Luis', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Luis! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of \\( 9 + 10 - 4 \\times 2^3 \\) is \\(-11\\).', additional_kwargs={}, response_metadata={})],
 'output': 'The result of \\( 9 + 10 - 4 \\times 2^3 \\) is \\(-11\\).'}

In [ ]:
9+10-(4*2)**3

-13

In [24]:
agent_executor.invoke({
    "input": "What is my name?",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Your name is Luis.

> Finished chain.


{'input': 'What is my name?',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Luis', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Luis! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of \\( 9 + 10 - 4 \\times 2^3 \\) is \\(-11\\).', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Luis.', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Luis.'}

### SerpAPI Weather Agent

In [33]:
import requests

from langchain.agents import load_tools
from dotenv import load_dotenv
from datetime import datetime

In [31]:
load_dotenv()

True

In [32]:
toolbox = load_tools(tool_names=['serpapi'], llm=llm)

In [38]:
@tool
def get_location_from_ip():
    """
    Get geographic location based on the IP address
    """
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        if 'loc' in data:
            latitude, longitude = data['loc'].split(',')
            data = (
                f"Latitude: {latitude}, \n"
                f"Longitude: {longitude}, \n"
                f"City: {data.get('city', 'N/A')}, \n"
                f"Country: {data.get('country', 'N/A')}, \n"
            )
            return data
        else:
            return "Location could not be determined"
    except Exception as e:
        return f"Error occurred: {e}"
    

@tool
def get_current_datetime() -> str:
    """
    return the current date and time
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [39]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [41]:
tools = toolbox + [get_current_datetime, get_location_from_ip]

In [42]:
agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

In [43]:
agent_executor_2 = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

In [44]:
out = agent_executor_2.invoke({
    "input": (
        "I have a few questions, what is the date and time right now?"
        "How is the weather where I am? Please give me degrees in Celsius"
    )
})



> Entering new AgentExecutor chain...

Invoking: `get_current_datetime` with `{}`


2025-08-11 19:14:16
Invoking: `get_location_from_ip` with `{}`


Latitude: -22.9706, 
Longitude: -46.9958, 
City: Valinhos, 
Country: BR, 

Invoking: `Search` with `Valinhos, Brazil weather`


{'type': 'weather_result', 'temperature': '57', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '62%', 'wind': '9 mph', 'location': 'Valinhos - State of São Paulo, Brazil', 'date': 'Monday 7:00 PM', 'weather': 'Clear'}The current date and time is **August 11, 2025, 19:14**.

The weather in **Valinhos, Brazil** is clear, with a temperature of **14°C** (converted from 57°F). The humidity is at **62%**, and there is no precipitation expected.

> Finished chain.


In [45]:
from IPython.display import display, Markdown

In [46]:
display(Markdown(out["output"]))

The current date and time is **August 11, 2025, 19:14**.

The weather in **Valinhos, Brazil** is clear, with a temperature of **14°C** (converted from 57°F). The humidity is at **62%**, and there is no precipitation expected.